In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import networkx as nx
import random

In [ ]:
def ricf_three_vars(L1, L2, L3, num_iter, var, max_degree_of_network):
    '''
    RICF stands for Residual Iterative Conditional Fitting, a method
    from the paper "Computing maximum likelihood estimates in recursive
    linear models with correlated errors" by Drton, Eichler, and Richardson.
    RICF is used to estimate the covariance matrix of a graphical model
    that specifies a multivariate normal distribution.

    This function estimates the covariance matrix of the joint distribution
    L1 <-> L2 <-> L3 (and also the edge L1 <-> L3),
    which is assumed to be generated from a multivariate normal distribution.

    Args:
        L1: a list of n independent realizations of L1.
        L2: a list of n independent realizations of L2.
        L3: a list of n independent realizations of L3.
        max_iter: number of iterations to run the optimization.
        var: the estimated, shared variance of L1, L2, and L3.
             var(L1) = var(L2) = var(L3) is by assumption.
        max_degree_of_network: largest degree among all vertices in the network.
            We need this information to ensure diagonal dominance throughout
            the optimization process, which is a sufficient condition for the
            positive definiteness of the estimated covariance matrix.

    Returns:
        A 2x2 numpy array representing the estimated covariance matrix of the
        joint distribution L1 <-> L2.
    '''

    def least_squares_loss(params, L, Z, var_index):
        n, d = L.shape
        params = list(params) * (d) # d x 1 vector, with same param repeated d times
        params = np.array(params)
        return 0.5 / n * np.linalg.norm(L[:, var_index] - np.dot(Z, params)) ** 2

    # de-mean every variable to get epsilons
    eps_L1 = L1 - np.mean(L1)
    eps_L2 = L2 - np.mean(L2)
    eps_L3 = L3 - np.mean(L3)

    L_df = pd.DataFrame({'L1': eps_L1, 'L2': eps_L2, 'L3': eps_L3})

    # random guess for cov mat
    # all these "0.0" are initial guesses and are shared parameters
    cov_mat = np.array([[var, 0.0, 0.0],
                        [0.0, var, 0.0],
                        [0.0, 0.0, var]])

    for _ in range(num_iter):
        for var_index in [0, 2]: # TODO: [0, 2]?
            omega = cov_mat.copy()
            omega_minusi = np.delete(omega, var_index, axis=0)
            omega_minusii = np.delete(omega_minusi, var_index, axis=1)
            omega_minusii_inv = np.linalg.inv(omega_minusii)

            epsilon = L_df.values
            epsilon_minusi = np.delete(epsilon, var_index, axis=1)

            Z_minusi = epsilon_minusi @ omega_minusii_inv.T
            Z = np.insert(Z_minusi, var_index, 0, axis=1)

            # bounds are to ensure positive definiteness of the covariance matrix
            # of the MVN that specify the joint distribution of p(L),
            # and we also add/minus a small constant in case the rounding goes
            # the wrong way
            bound = (-var/float(max_degree_of_network) + 1e-10,
                      var/float(max_degree_of_network) - 1e-10)

            # getting the solution from five random initializations
            # and pick the one with the smallest loss
            best_solution = None
            best_loss = np.inf
            for _ in range(5):
                # minimize by first setting a random start within the bounds
                sol = minimize(least_squares_loss,
                               x0=np.random.uniform(low=bound[0], high=bound[1], size=1), # a vector of size 1 because there's only 1 free param
                               args=(L_df.values, Z, var_index),
                               method='L-BFGS-B',
                               bounds=[bound])
                if sol.fun < best_loss:
                    best_loss = sol.fun
                    best_solution = sol

            # update covariance matrix according to the best solution
            # ORIGINAL LINE: cov_mat[:, var_index] = cov_mat[var_index, :] = best_solution.x[0]
            # ASK ROHIT: SHOULD I BE UPADTING EVERY (SHARED) PARAMETER IN cov_mat?
            # a mask for non-diagonal elements
            mask = ~np.eye(cov_mat.shape[0], dtype=bool)
            cov_mat[mask] = best_solution.x[0]

            # this is a trivial update for graphs with only bidirected edges
            cov_mat[0, 0] = cov_mat[1, 1] = var

    return cov_mat

In [ ]:
true_cov_mat = np.array([[2, 0.8, 0.8],
                         [0.8, 2, 0.8],
                         [0.8, 0.8, 2]])
L = np.random.multivariate_normal([0,0,0], true_cov_mat, size=100000)
L1 = L[:,0]
L2 = L[:,1]
L3 = L[:,2]
print(np.var(L1), np.var(L2), np.var(L3))

2.0035273486400897 1.9951242170948238 2.0202328880778464


In [ ]:
ricf_three_vars(L1, L2, L3, 20, 2, 2)

array([[2.        , 0.80354466, 0.80354466],
       [0.80354466, 2.        , 0.80354466],
       [0.80354466, 0.80354466, 2.        ]])

In [ ]:
# thoughts
# pass in an adjacency matrix, use it as a mask